In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [3]:
train_dir = "/content/drive/MyDrive/Cattle Classification/data/train"
val_dir = "/content/drive/MyDrive/Cattle Classification/data/val"
test_dir = "/content/drive/MyDrive/Cattle Classification/data/test"

IMG_SIZE = 224
BATCH_SIZE = 32

In [4]:
# data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range = 15,
    horizontal_flip = True,
    width_shift_range=0.10,
    height_shift_range=0.10,
    zoom_range = 0.2,
    fill_mode = "nearest"
)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    shuffle = True
)

# no augmentation
val_datagen = ImageDataGenerator(preprocessing_function= preprocess_input)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    shuffle = False
)

# no augmentation
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    shuffle = False
)

Found 965 images belonging to 5 classes.
Found 119 images belonging to 5 classes.
Found 124 images belonging to 5 classes.


In [5]:
base_model = MobileNetV2(include_top = False, weights = "imagenet", input_shape = (IMG_SIZE, IMG_SIZE, 3))

for layer in base_model.layers:
  layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.4)(x)
output = Dense(5, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

# compile with Adam optimizer
model.compile(
    optimizer = Adam(learning_rate=0.001),
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

if __name__ == "__main__":

  history = model.fit(
    train_data,
    validation_data = val_data,
    epochs = 30,
    callbacks = [early_stop],
    verbose = 1
  )

  # save trained model
  model.save("/content/drive/MyDrive/Cattle Classification/saved_models/mobilenet_v2.h5")

  with open("/content/drive/MyDrive/Cattle Classification/saved_models/history_mobilenet_v2.pkl", "wb") as f:
    pickle.dump(history.history, f)

test_loss, test_acc = model.evaluate(test_data, verbose=1)
print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 650s 21s/step - accuracy: 0.4498 - loss: 1.5211 - val_accuracy: 0.7227 - val_loss: 0.7140
Epoch 2/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 476ms/step - accuracy: 0.7353 - loss: 0.7235 - val_accuracy: 0.7395 - val_loss: 0.7184
Epoch 3/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 482ms/step - accuracy: 0.8086 - loss: 0.5895 - val_accuracy: 0.8319 - val_loss: 0.4882
Epoch 4/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 482ms/step - accuracy: 0.8077 - loss: 0.5218 - val_accuracy: 0.8151 - val_loss: 0.4924
Epoch 5/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 492ms/step - accuracy: 0.8259 - loss: 0.4888 - val_accuracy: 0.8571 - val_loss: 0.4585
Epoch 6/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 492ms/step - accuracy: 0.8356 - loss: 0.4494 - val_accuracy: 0.8151 - val_loss: 0.5243
Epoch 7/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 505ms/step - accuracy: 0.8735 - loss: 0.3700 - val_accuracy: 0.8571 - val_loss: 0.4428
Epoch 8/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 498ms/step - accuracy: 0.8895 - loss: 0.3211 - val_accur

4/4 ━━━━━━━━━━━━━━━━━━━━ 53s 17s/step - accuracy: 0.8357 - loss: 0.5974
Test Loss: 0.4786 | Test Acc: 0.8548
